# A Parallel workflow LLM app
Here we are attempting to create a LLM application with parallel workflow

### 1. Import dependencies

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama.chat_models import ChatOllama
from typing import TypedDict

In [ ]:
# For graphical visualization of the state graph
 
import nest_asyncio
nest_asyncio.apply()
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import Image

### 2. Linking the model

In [ ]:
model = ChatOllama(model="gemma:2b")

### 3. Create a state

In [ ]:
class BlogState(TypedDict):                      # The BlogState inherits a TypedDict
    topic: str
    outline: str
    blog: str

### 4. Define the function for the graph node

In [ ]:
def llm_outline(state: BlogState) -> BlogState:
    # Extract the question from the state
    topic = state["topic"]


    # Form a prompt
    prompt = f"I require an outline for a blog on {topic}."


    # Ask the question to the model
    outline = model.invoke(prompt).content


    # Update the state with the answer
    state['outline'] = outline

    return state


def llm_blog(state: BlogState) -> BlogState:
    # Extract the outline from the state
    outline = state["outline"]


    # Form a prompt
    prompt = f"Write a detailed blog based on the following outline:\n{outline}"


    # Ask the question to the model
    blog = model.invoke(prompt).content


    # Update the state with the answer
    state["blog"] = blog

    return state

### 5. Create the graph

In [ ]:
# Declare the graph:
graph = StateGraph(BlogState)                    # The StateGraph takes the state as a parameter, signifying that the "state" will be exchanged between the nodes through the edges.

# Add a nodes/states to the graph:
graph.add_node("llm_outline", llm_outline)
graph.add_node("llm_blog", llm_blog)


# Add edges to the graph:
graph.add_edge(START, 'llm_outline')
graph.add_edge('llm_outline', 'llm_blog')
graph.add_edge('llm_blog', END)

### 6. Compile the graph

In [ ]:
workflow = graph.compile()

### 7. Run the code

In [ ]:
initial_state = { "topic" : "Motorcycles" }

final_state = workflow.invoke(initial_state)

print(final_state)


In [ ]:
print(final_state["outline"])

In [ ]:
print(final_state["blog"])

### 8. The Graph

In [ ]:
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER))